In [1]:
z

import os
import torch
import torchvision
import matplotlib.pyplot as plt

from torch.utils.data import Dataset
from ochumanApi.ochuman import Poly2Mask
from torch.utils.data import Dataset
from ochumanApi.ochuman import OCHuman


%matplotlib inline
plt.rcParams['figure.figsize'] = (15, 15)
torch.manual_seed(1)

  Cloning https://github.com/facebookresearch/detectron2.git to /private/var/folders/_t/s7kkvk9540n3cp76z802ksnh0000gp/T/pip-req-build-t18dzrr4
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /private/var/folders/_t/s7kkvk9540n3cp76z802ksnh0000gp/T/pip-req-build-t18dzrr4
  Resolved https://github.com/facebookresearch/detectron2.git to commit e9f7e2ba15abd7badcb05ef6f5076f06b36a9c5b
  Preparing metadata (setup.py) ... done


In [6]:
import numpy as np 

from torchvision.io import read_image, ImageReadMode
from torchvision.ops.boxes import masks_to_boxes
from torchvision.tv_tensors import BoundingBoxes, Mask
from torchvision import tv_tensors
from torchvision.transforms.v2 import functional as F, Transform


class OCHumanDataset(Dataset):
    def __init__(self, image_root: str, oc_human_object: OCHuman, transforms):
        self.root = image_root
        self.data = oc_human_object.loadImgs(imgIds=oc_human_object.getImgIds())
        self.images, self.masks = self.__get_properties(oc_human_data=self.data)
        self.transforms = transforms
        self.__getitem__(2)
        
    def __getitem__(self, index):
        # image = self.images[index]/255
        # masks = self.masks[index]
        # masks = torch.as_tensor(masks, dtype=torch.uint8)
        # 
        # number_of_objects = len(masks)
        # 
        # labels = torch.ones(number_of_objects, dtype=torch.int64)
        # 
        # bounding_boxes = torch.as_tensor(self.bounding_boxes[index][:])
        # area = torch.as_tensor(self.__get_areas(index))
        # 
        # image_id = index
        # 
        # is_crowd = torch.zeros(number_of_objects, dtype=torch.int64)
        # 
        # canvas_size = image.size()
        # 
        # target = {'boxes': BoundingBoxes(bounding_boxes, format='XYXY', canvas_size=canvas_size[1:]),
        #           'masks': Mask(masks), 'labels': labels, 'image_id': image_id, 'area': area,
        #           'iscrowd': is_crowd}
        image = self.images[index]/255
        masks = self.masks[index]
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        number_of_objects = len(masks)

        labels = torch.ones(number_of_objects, dtype=torch.int64)

        bounding_boxes = masks_to_boxes(masks=masks)
        area = (bounding_boxes[:, 3] - bounding_boxes[:, 1]) * (bounding_boxes[:, 2] - bounding_boxes[:, 0])

        image_id = torch.as_tensor(index, dtype=torch.int64)

        is_crowd = torch.zeros(number_of_objects, dtype=torch.uint8)

        target = {}
        target["boxes"] = tv_tensors.BoundingBoxes(bounding_boxes, format="XYXY", canvas_size=F.get_size(image))
        target["masks"] = tv_tensors.Mask(masks)
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = is_crowd

        if self.transforms is not None:
            img, target = self.transforms(image, target)

        return image, target
    
    def __len__(self):
        return len(self.images)
    
    def __get_properties(self, oc_human_data):
        images = []
        image_masks = []
        for file in oc_human_data:
            images.append(read_image(os.path.join(self.root, file['file_name']), mode=ImageReadMode.RGB))
            image_masks.append(self.__get_binary_masks(file=file))
        return images, image_masks
    
    @staticmethod
    def __get_binary_masks(file):
        masks = []
        for annotation in file['annotations']:
            segmentation = annotation['segms']
            if segmentation is not None:
                masks.append(Poly2Mask(segmentation))
        return masks

In [7]:
import utils

from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from engine import train_one_epoch, evaluate
from tqdm import trange
from datetime import datetime

oc_human = OCHuman(AnnoFile='./ochuman.json', Filter='kpt&segm')

In [8]:
from torchvision.transforms import v2 as T


def get_transform(train):
    transforms = []
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    transforms.append(T.ToDtype(torch.float, scale=True))
    transforms.append(T.ToPureTensor())
    return T.Compose(transforms)

dataset = OCHumanDataset(image_root='images/', oc_human_object=oc_human, transforms=get_transform(train=True))

/var/folders/_t/s7kkvk9540n3cp76z802ksnh0000gp/T/ipykernel_20420/756242265.py:41: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:264.)
  masks = torch.as_tensor(masks, dtype=torch.uint8)


In [4]:
from google.colab.patches import cv2_imshow
masks = dataset[17][1]['masks']
for mask in masks:
    cv2_imshow(mask.numpy() * 255)

/Users/rouna/anaconda3/envs/thirdlama/lib/python3.9/site-packages/google/colab/data_table.py:30: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  from IPython.utils import traitlets as _traitlets


NameError: name 'dataset' is not defined

In [ ]:

indices = torch.randperm(len(dataset)).tolist()

dataset_train = torch.utils.data.Subset(dataset, indices[:5])

dataset_validation = torch.utils.data.Subset(dataset, indices[3001:3002])

dataset_test = torch.utils.data.Subset(dataset, indices[4001:4002])

data_loader = torch.utils.data.DataLoader(
    dataset_train,
    batch_size=2,
    shuffle=True,
    collate_fn=utils.collate_fn
)

data_loader_validation = torch.utils.data.DataLoader (
    dataset_validation,
    batch_size=2,
    shuffle=False,
    collate_fn=utils.collate_fn
)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test,
    batch_size=2,
    shuffle=False,
    collate_fn=utils.collate_fn
)

In [4]:
import torchvision
import torch
model = torchvision.models.detection.maskrcnn_resnet50_fpn_v2(weights='DEFAULT')

In [5]:
number_of_classes = 2

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

print(model.roi_heads.maskrcnn_loss)

# in_features = model.roi_heads.box_predictor.cls_score.in_features
# 
# model.roi_heads.box_predictor = FastRCNNPredictor(in_features, number_of_classes)
# 
# in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
# hidden_layer = 256
#     
# model.roi_heads.mask_predictor = MaskRCNNPredictor(
#     in_features_mask,
#     hidden_layer,
#     number_of_classes,
# )

AttributeError: 'RoIHeads' object has no attribute 'maskrcnn_loss'

In [9]:
params = [p for p in model.parameters() if p.requires_grad]

num_epochs = 100

optimizer = torch.optim.SGD(
    params,
    lr=0.005,
    momentum=0.9,
    weight_decay=0.0005
)

# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer,
    step_size=3,
    gamma=0.1
)

def get_current_time():
    now = datetime.now()
    now = now.strftime("%b-%d-%Y_%H-%M-%S")
    return now

def save_model(current_model, time, directory):
    model_scripted = torch.jit.script(current_model) 
    model_scripted.save(directory + '/checkpoint_' + time + '.pt')
    
filename = "run_" + get_current_time()
run_directory = "/storage/runs/" + filename
os.mkdir(run_directory)

evaluate(model, data_loader_validation, device=device)

for epoch in trange(num_epochs, desc='Training Epoch'):
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=500)
    save_model(model, get_current_time(), run_directory)
    lr_scheduler.step()
    with torch.no_grad():
        evaluate(model, data_loader_validation, device=device)
    
with torch.no_grad():
    evaluate(model, data_loader_test, device=device)

AttributeError: 'Tensor' object has no attribute 'astype'